In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
#from langchain_mistralai import ChatMistralAI
from langchain_google_genai import ChatGoogleGenerativeAI

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [ ]:
from langchain.agents import create_agent

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash") 

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt,
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [11]:
from pprint import pprint

pprint(response["messages"][-1].content)

[{'extras': {'signature': 'Co8JAXLI2ny4Es7Hms5EmF8Ggj7S/wJAW6wNfOkzmN81BjITqTGc9lWa3nEs5v5aoS5g4EVZJW1iCKY13fmosPVuGDvx8NojQiBr1u/zU2DNSjatbIbAyZdrf90nwMve9KtzmRaYNG4UIcFRSUX8e7nxWrFEmQVTrQ3IcLtBE4ltxbtHvI86B7llGec2tdRp/q2Y1EhtcnBcDCGFHzOnNTEHBhYs76vPiDhJoWmG0TEnwkawBF0N5jG84p82SeCG1nT2C5/ZhsUlP5cfOEFcdnqhY2ymhRnukUAZq/ot1M4we6hrRKTxjsQTlu+tbhdZ3E+na7TDXaSUfzoIkoDmJBTdA5IIqwynhWw+xHakbQX7367doPJ8SGAwGW9tZ7w3oL20r9o9nUFZL/6+vyI6DyKMfkN2bmyUuc9sINGohfUB4s1xlKL8LkKnLRWQpFaXapaxPSlHjqSDIyfntfDU+bUx3xceR9PxcOOEFPVJKhCN/HF7LDFhcnwgsp99kAiup7Z60h33RmWJIWEYD5BRtPgtezvYN3l1WRvLlreHBRMLa/zYkSqgdKAVxo7xz3LFzUwp8gysC6gevB3PGtaHSMcKd1s9Fy+8XuYf4TXusHGQ9bIEU1BeO+iOM49QAWbRjbiYlZQg7UE3WMP7UiUYKlNhefcZBpyOovd3jHjVEzYe1i4XTc9uF52BrwIOJ2CBngOuJE/FWnZemgikHUWvrE6YyrJqLNpimCdFoF+PhxPZv0jmsjqBgaNtQ6w4HHNMGpYx0yyxSWRTEj1r066o4HD9ZtRGNcIq/N/70t1wMwaYv3tYoMaGCJs2E0Tux+cjoHP5BJYMlUGgL2hzVq3rxhLPUhmj05rh8XZ9CxgOjr1PQNIq571Z/+EigNNhnUVzz9u5D/OKAQMyKCB/ejZg/Yo6CmeEiPUqHD/w86HK9s4Z+kfjdQcnzfplE9M25uoN/4blwxEWVCNRQ

## Online MCP

In [ ]:
client = MultiServerMCPClient( # for the mcp, I can look for any mcp in the web, then get this json structure, and create a client for it
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [19]:
tools

[StructuredTool(name='get_current_time', description='Get current time in a specific timezones', args_schema={'type': 'object', 'properties': {'timezone': {'type': 'string', 'description': "IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'America/New_York' as local timezone if no timezone provided by the user."}}, 'required': ['timezone']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001EBE3CE7600>),
 StructuredTool(name='convert_time', description='Convert time between timezones', args_schema={'type': 'object', 'properties': {'source_timezone': {'type': 'string', 'description': "Source IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'America/New_York' as local timezone if no source timezone provided by the user."}, 'time': {'type': 'string', 'description': 'Time to convert in 24-hour format (HH:MM)'}, 'target_timezone': {'type': 'string', 'description': "Target IANA ti

In [ ]:
agent = create_agent(
    #model="gpt-5-nano",
    model, # "gemini-flash-2.5"
    tools=tools,
)

In [18]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='c2d4f26a-08d3-4a1b-ad48-3c87380f5670'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"timezone": "America/New_York"}'}, '__gemini_function_call_thought_signatures__': {'cdd1548e-ec8b-4e80-97dd-e9c29404a5e5': 'CrwCAXLI2ny+90hF1p1QLX2uQtkngmFGFG/CmJmTNWYbHQ/IdP0AuxJgLfGQOCNwrUQOuHNnI5dkQIinZG/axKlw79gwuMw37SHTasPhRj50gXnrKdtrvQ4hPznXj4CxjgFU264tFIAADCAX7lF2zF63KbqLkraO1A3ivwTizz5YOSdPLtN7OS/QHky2YQJ75bYKUAL5ahXoMp55KcaDCubeJff20W/zznqt/mmrOf4UiCg5il3bNRfbVRXC5T2Q41aAXedy0HYKdCvioF/81iD09C/Gk8i31cRmAhRJzd7VsF+LghqSo9cO09oFgCqSuOBQbd0G+4QRyXZvrmyZjRToYA8a8OLhXxDKD2uw7fkQMGAUiLzj22LLIzj6RVbBacuYmBe3pI0NU1aZrt0+xCW5CXKa14TDk+PvdhJnog=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9804-e358-7712-80ec-2e7a72b4945e-